In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np
import time


# Load MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# Open webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Smoothing parameters
prev_gesture = None
gesture_count = 0
SMOOTHING_THRESHOLD = 5  # consecutive frames

# Slope threshold for “straight”
STRAIGHT_THRESHOLD = 0.3

# Key mapping
KEYS = {
    'left': 'a',
    'right': 'd',
    'straight': 'w',
    'stop': 's'
}


while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    # Process with MediaPipe
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    # Default to stop
    gesture = 'stop'

    # If two hands detected, compute slope between their middle-finger tips
    if results.multi_hand_landmarks and len(results.multi_hand_landmarks) >= 2:
        pts = []
        for handLms in results.multi_hand_landmarks:
            lm = handLms.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
            pts.append((lm.x * w, lm.y * h))
            mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)

        # Sort so pts[0] is left hand, pts[1] is right hand (by x-coordinate)
        pts = sorted(pts, key=lambda p: p[0])
        (x1, y1), (x2, y2) = pts[:2]

        slope = (y2 - y1) / (x2 - x1 + 1e-6)

        if abs(slope) < STRAIGHT_THRESHOLD:
            gesture = 'straight'
        elif slope > 0:
            gesture = 'right'
        else:
            gesture = 'left'
    # else: gesture remains 'stop'

  
    if gesture == prev_gesture:
        gesture_count += 1
    else:
        gesture_count = 0
        prev_gesture = gesture

    if gesture_count >= SMOOTHING_THRESHOLD:
        # Release all keys firsts
        for k in KEYS.values():
            pyautogui.keyUp(k)
        # Press the new key
        pyautogui.keyDown(KEYS[gesture])
        gesture_count = 0


    cv2.putText(frame, f"Gesture: {gesture}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Steering Control", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


C:\Users\kavit\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
